In [1]:
##
import sys
import os
import pickle
import sys
import warnings
import enum
from enum import Enum
import concurrent
from typing import Optional, List, Dict, Callable

##
import matplotlib
from pathlib import Path

##
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

##
from finrl import config as finrl_config
from finrl.main import check_and_make_directories
from finrl.config_tickers import (
    SINGLE_TICKER,
    DOW_30_TICKER,
    NAS_100_TICKER,
    SP_500_TICKER,
    HSI_50_TICKER,
    SSE_50_TICKER,
    CSI_300_TICKER,
    CAC_40_TICKER,
    DAX_30_TICKER,
    TECDAX_TICKER,
    MDAX_50_TICKER,
    SDAX_50_TICKER,
    LQ45_TICKER,
    SRI_KEHATI_TICKER,
    FX_TICKER,
)

In [2]:
def config():
    # #
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    # os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # INFO, WARN are ignored and only ERROR messages will be printed

    # #
    matplotlib.use("Agg")

    # #
    check_and_make_directories(
        [
            finrl_config.DATA_SAVE_DIR,
            finrl_config.TRAINED_MODEL_DIR,
            finrl_config.TENSORBOARD_LOG_DIR,
            finrl_config.RESULTS_DIR,
        ]
    )


config()

In [3]:
##
import pandas as pd
import tqdm
from dotenv import load_dotenv
import financedatabase as fd
import fundamentalanalysis as fa

##
from common.utils import now_time
from configuration.settings import ProjectDir
from rl.data.financial_modeling_prep import download
from rl.data.Company import Company

In [4]:
prj_dir = ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing"))

In [5]:
load_dotenv(prj_dir.root.joinpath("env/.env"))
try:
    print(os.getenv("EOD_HISTORICAL_DATA_API"))
    print(os.getenv("NASDAQ_HISTORICAL_DATA_API"))
    print(os.getenv("ALPHA_VANTAGE_API"))
    print(os.getenv("FINANCIAL_MODELING_PREP_API"))
except:
    raise ValueError("ERROR: API keys")

639b4cc5af44e7.93519913
B9p3RKb-K6xWwBRLso7Z
G21Y7NB8U8VJFFIH
6b0006554aa86e3e21493713af2af3c4


In [6]:
def get_download_range(_from: int, _plus_costant: int) -> tuple:
    return (_from, _from + _plus_costant)


def download_subprocess(start: int, stop: int, step: int, companies: List[str]) -> dict:
    all_symbols_info = dict()
    with concurrent.futures.ProcessPoolExecutor() as executor:
        result = [
            executor.submit(
                download, companies[get_download_range(i, step)[0] : get_download_range(i, step)[1]], api_key
            )
            for i in range(start, stop, step)
        ]

        for i in concurrent.futures.as_completed(result):
            try:
                all_symbols_info.update(i.result())
            except Exception as e:
                print(e)
    return all_symbols_info

In [7]:
#
api_key = os.getenv("FINANCIAL_MODELING_PREP_API")

In [8]:
#
indexes: List[List[str]] = [
    # DOW_30_TICKER,
    # NAS_100_TICKER,
    SP_500_TICKER,
    HSI_50_TICKER,
    SSE_50_TICKER,
    CSI_300_TICKER,
    CAC_40_TICKER,
    DAX_30_TICKER,
    TECDAX_TICKER,
    MDAX_50_TICKER,
    SDAX_50_TICKER,
    LQ45_TICKER,
    SRI_KEHATI_TICKER,
    FX_TICKER,
]

In [9]:
start = 0
stop = indexes.__len__()
step = 200
processes = 4

for tickers in indexes:
    ##
    _start = 0
    _stop = tickers.__len__()
    _step = _stop // processes
    print(tickers)
    print(f"{_start}, {_stop}, {_step}")

    # get_download_range()
    print([get_download_range(i, _step) for i in range(_start, _stop, _step)])

    ##
    filename = prj_dir.dataset.financial_modeling_prep.joinpath(f"{_start}_{_stop}_companies_{now_time()}.pkl")
    print(filename)
    downloaded_tickers: dict = download_subprocess(_start, _stop, _step, tickers)
    with open(file=filename, mode="wb") as f:
        import pickle

        pickle.dump(downloaded_tickers, f)

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADS', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AGN', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMG', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ARNC', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBT', 'BBY', 'BDX', 'BEN', 'BF.B', 'BHGE', 'BIIB', 'BK', 'BKNG', 'BLK', 'BLL', 'BMY', 'BR', 'BRK.B', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CBOE', 'CBRE', 'CBS', 'CCI', 'CCL', 'CDNS', 'CE', 'CELG', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'COTY', 'CPB', 'CPRI', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTL', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR'

Symbol: CTSH: 100%|██████████| 125/125 [32:02<00:00, 15.38s/it]


['0011.HK', '0005.HK', '0012.HK', '0006.HK', '0003.HK', '0016.HK', '0019.HK', '0002.HK', '0001.HK', '0267.HK', '0101.HK', '0941.HK', '0762.HK', '0066.HK', '0883.HK', '2388.HK', '0017.HK', '0083.HK', '0939.HK', '0388.HK', '0386.HK', '3988.HK', '2628.HK', '1398.HK', '2318.HK', '3328.HK', '0688.HK', '0857.HK', '1088.HK', '0700.HK', '0836.HK', '1109.HK', '1044.HK', '1299.HK', '0151.HK', '1928.HK', '0027.HK', '2319.HK', '0823.HK', '1113.HK', '1038.HK', '2018.HK', '0175.HK', '0288.HK', '1997.HK', '2007.HK', '2382.HK', '1093.HK', '1177.HK', '2313.HK']
0, 50, 12
[(0, 12), (12, 24), (24, 36), (36, 48), (48, 60)]
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/financialmodelingprep/0_50_companies_2022-12-17T14-46-14.pkl


Symbol: 1398.HK: 100%|██████████| 12/12 [01:23<00:00,  6.97s/it]


['600000.XSHG', '600036.XSHG', '600104.XSHG', '600030.XSHG', '601628.XSHG', '601166.XSHG', '601318.XSHG', '601328.XSHG', '601088.XSHG', '601857.XSHG', '601601.XSHG', '601668.XSHG', '601288.XSHG', '601818.XSHG', '601989.XSHG', '601398.XSHG', '600048.XSHG', '600028.XSHG', '600050.XSHG', '600519.XSHG', '600016.XSHG', '600887.XSHG', '601688.XSHG', '601186.XSHG', '601988.XSHG', '601211.XSHG', '601336.XSHG', '600309.XSHG', '603993.XSHG', '600690.XSHG', '600276.XSHG', '600703.XSHG', '600585.XSHG', '603259.XSHG', '601888.XSHG', '601138.XSHG', '600196.XSHG', '601766.XSHG', '600340.XSHG', '601390.XSHG', '601939.XSHG', '601111.XSHG', '600029.XSHG', '600019.XSHG', '601229.XSHG', '601800.XSHG', '600547.XSHG', '601006.XSHG', '601360.XSHG', '600606.XSHG', '601319.XSHG', '600837.XSHG', '600031.XSHG', '601066.XSHG', '600009.XSHG', '601236.XSHG', '601012.XSHG', '600745.XSHG', '600588.XSHG', '601658.XSHG', '601816.XSHG', '603160.XSHG']
0, 62, 15
[(0, 15), (15, 30), (30, 45), (45, 60), (60, 75)]
/Users/zl

...

Symbol: UNVR.JK: 100%|██████████| 11/11 [01:16<00:00,  6.92s/it]


['AALI.JK', 'ADHI.JK', 'ASII.JK', 'BBCA.JK', 'BBNI.JK', 'BBRI.JK', 'BBTN.JK', 'BMRI.JK', 'BSDE.JK', 'INDF.JK', 'JPFA.JK', 'JSMR.JK', 'KLBF.JK', 'PGAS.JK', 'PJAA.JK', 'PPRO.JK', 'SIDO.JK', 'SMGR.JK', 'TINS.JK', 'TLKM.JK', 'UNTR.JK', 'UNVR.JK', 'WIKA.JK', 'WSKT.JK', 'WTON.JK']
0, 25, 6
[(0, 6), (6, 12), (12, 18), (18, 24), (24, 30)]
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/financialmodelingprep/0_25_companies_2022-12-17T15-00-21.pkl


Symbol: JSMR.JK: 100%|██████████| 6/6 [00:39<00:00,  6.56s/it]


['AUDCAD=X', 'AUDCHF=X', 'AUDJPY=X', 'AUDNZD=X', 'AUDSGD=X', 'AUDUSD=X', 'AUDUSD=X', 'AUDUSD=X', 'AUDUSD=X', 'AUDUSD=X', 'AUDUSD=X', 'AUDUSD=X', 'CADCHF=X', 'CADHKD=X', 'CADJPY=X', 'CHFJPY=X', 'CHFSGD=X', 'EURAUD=X', 'EURCAD=X', 'EURCHF=X', 'EURCHF=X', 'EURCHF=X', 'EURCZK=X', 'EURGBP=X', 'EURHKD=X', 'EURHUF=X', 'EURJPY=X', 'EURNOK=X', 'EURNZD=X', 'EURPLN=X', 'EURRUB=X', 'EURSEK=X', 'EURSGD=X', 'EURTRY=X', 'EURTRY=X', 'EURUSD=X', 'GBPAUD=X', 'GBPAUD=X', 'GBPAUD=X', 'GBPCAD=X', 'GBPCHF=X', 'GBPJPY=X', 'GBPNZD=X', 'GBPUSD=X', 'HKDJPY=X', 'NZDCAD=X', 'NZDCHF=X', 'NZDJPY=X', 'NZDUSD=X', 'SGDJPY=X', 'TRYJPY=X', 'USDCAD=X', 'USDCHF=X', 'USDCNH=X', 'USDCZK=X', 'USDHKD=X', 'USDHUF=X', 'USDILS=X', 'USDJPY=X', 'USDMXN=X', 'USDNOK=X', 'USDPLN=X', 'USDRON=X', 'USDRUB=X', 'USDSEK=X', 'USDSGD=X', 'USDTHB=X', 'USDTRY=X', 'USDZAR=X', 'XAGUSD=X', 'XAUUSD=X', 'ZARJPY=X', 'EURDKK=X']
0, 73, 18
[(0, 18), (18, 36), (36, 54), (54, 72), (72, 90)]
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/finan

Symbol: EURAUD=X: 100%|██████████| 18/18 [00:53<00:00,  2.99s/it]
